In [ ]:
from semanticscholar import SemanticScholar
import csv
import pandas as pd
import random
import json
import shutil
import numpy as np

import os
from neo4j import GraphDatabase


In [ ]:
NEO4J_URI = "bolt://localhost:7687"  # Default for Neo4j Desktop
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pass1234"  # Change this to your actual password
IMPORT_FOLDER = os.path.expanduser("~/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-6f9440a9-c56d-4ebc-b244-90d92b771350/import/")

In [ ]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

session = driver.session() 

1. Node Similarity Algorithm: For Paper Similarity Based on Topics 

In [ ]:
# drop projected graph if exists

query = """CALL gds.graph.drop('paperTopicGraph', false)"""
result = session.run(query)

print([record for record in result])


In [ ]:
# Check similarity (project the graph, run algorith, and retrun similariy)

query = """
        CALL gds.graph.project(
        'paperTopicGraph',
        ['Paper', 'Keyword'],
        {
            About: {
            type: 'About',
            orientation: 'UNDIRECTED'
            }
        }
        )
        """
result = session.run(query)

print(list(result))

In [ ]:
query = """

        CALL gds.nodeSimilarity.stream('paperTopicGraph')
        YIELD node1, node2, similarity
        RETURN 
        gds.util.asNode(node1).DOI AS Paper1_DOI,
        gds.util.asNode(node2).DOI AS Paper2_DOI,
        similarity
        ORDER BY similarity DESC
        Limit 10;

        """
result = session.run(query)

print(list(result))

2. PageRank: Identifying the most influential papers in the network considering citations.

This will help determine which papers are most influential based on how many other papers cite them and how important those citing papers are.

In [ ]:
# drop projected graph if exists

query = """CALL gds.graph.drop('citationGraph', false)"""
result = session.run(query)

print([record for record in result])

In [ ]:
# Project the graph first
query = """
CALL gds.graph.project(
  'citationGraph',
  'Paper',
  {
    Cited_by: {
      type: 'Cited_by',
      orientation: 'REVERSE'
    }
  }
)

"""

result = session.run(query)
print([record for record in result])


In [ ]:

# Run weighted PageRank
query = """
CALL gds.pageRank.stream('citationGraph')
YIELD nodeId, score
MATCH (p:Paper) WHERE id(p) = nodeId
RETURN p.DOI AS paper, score
ORDER BY score desc
LIMIT 10
"""

result = session.run(query)
print([record for record in result])